# Homework Session 11 Neural networks

### part 1

The purpose of the below is to classify days over years 2017-2018 by their corresponding mobility patterns between 10 zones in Taipei (quantified by an aggregated temporal network of subway ridership flows across the city)

In [84]:
import numpy as np
import pandas as pd
import keras
from keras.datasets import mnist
from keras.models import Model, Sequential
from keras.layers import Input, Dense
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from sklearn.preprocessing import MinMaxScaler
from keras.utils import np_utils
from keras import backend as K
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

import numpy as np
from nltk.tokenize import RegexpTokenizer
from keras.models import Sequential, load_model
from keras.layers import LSTM
from keras.layers.core import Dense, Activation
from tensorflow.keras.optimizers import RMSprop
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import pickle
import heapq

In [148]:
#read the data
TNet=pd.read_csv('taipeiD_TNet2.csv',header=None);

In [149]:
TNet.head() 
#each row represents a 10x10 adjacency matrix of the normalized Taipei subway mobility network between 10 zones flattened into a 100x1 row corresponding to a single day
#days start at jan-1-2017

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,0.017943,0.005415,0.003590,0.008316,0.007859,0.012942,0.012196,0.019543,0.001196,0.003327,...,0.002529,0.001533,0.001860,0.002375,0.005408,0.008922,0.003945,0.011075,0.005073,0.012708
1,0.021283,0.005215,0.003530,0.009359,0.007803,0.014288,0.011185,0.019044,0.001383,0.003499,...,0.002803,0.001757,0.001783,0.002549,0.005515,0.009650,0.003596,0.009618,0.005946,0.013709
2,0.028988,0.006511,0.005591,0.012970,0.007816,0.015878,0.010973,0.015768,0.002252,0.005388,...,0.004649,0.002555,0.002672,0.004291,0.007385,0.009558,0.004293,0.008791,0.010040,0.016301
3,0.029534,0.006471,0.005615,0.013017,0.007717,0.016098,0.011182,0.015815,0.002325,0.005443,...,0.004611,0.002473,0.002635,0.004195,0.007255,0.009487,0.004316,0.008729,0.010296,0.016437
4,0.029333,0.006525,0.005727,0.013098,0.007692,0.016358,0.011000,0.015677,0.002344,0.005527,...,0.004694,0.002515,0.002677,0.004222,0.007269,0.009921,0.004387,0.008923,0.010381,0.016914


In [150]:
#convert to an array and scale the data
X=np.array(TNet);

In [151]:
X=MinMaxScaler(feature_range=(0, 1), copy=True).fit_transform(X)

In [152]:
X.shape

(669, 100)

In [153]:
#define day of the week corresponding to each day of observation; 0-Sunday, 1-Monday,...,6-Saturday
y=np.array(range(669))%7; y[:10]

array([0, 1, 2, 3, 4, 5, 6, 0, 1, 2])

In [154]:
yc=np_utils.to_categorical(y) #get categorical binary variables isSunday, isMonday,...
yc[:5]

array([[1., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0.]], dtype=float32)

In [155]:
X_test=X[400:,:]; X_train=X[:400,:]; #split the data into training and test
y_test=yc[400:,:]; y_train=yc[:400,:]

## Task 1. Classify weekdays/weekends
Label the rows with ones for weekends, zeros for weekdays.
Train a neural network with 4 layers of 30,10,3 and 1 (output) neurons over the training sample against this label, evaluating its performance over the test sample. Report the acheived accuracy (categorical) over the test sample

First three layers use relu activation function, last one - sigmoid.
Use loss='binary_crossentropy', optimizer='adam', 100 epochs, batch_size=20. 

In [93]:
a = np.empty([669,2], dtype = int) 
for i in range(len(yc)):
    sum = 0
    for j in range(5):
        sum+= yc[i,j]
    a[i,0] = sum
    sum2 = 0
    for j in range(5,7):
        sum2+= yc[i,j]
    a[i,1] = sum2
    
b = np.empty([669,1], dtype = int) 
for i in range(len(a)):  
    if a[i,0]==1:
        b[i,0] = 1
    else:
        b[i,0] = 0
    
X_test=X[400:,:]; X_train=X[:400,:]
y_test=b[400:,:]; y_train=b[:400,:]        

In [94]:
dim = X_train.shape[1]
model = Sequential()
model.add(Dense(30, activation='relu', input_dim=dim))
model.add(Dense(10, activation='relu'))
model.add(Dense(3, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, batch_size=20, verbose=2)


# test['dl_predict'] = model.predict(X_test)

Epoch 1/100
20/20 - 1s - loss: 0.7070 - accuracy: 0.5475 - val_loss: 0.6892 - val_accuracy: 0.7175 - 921ms/epoch - 46ms/step
Epoch 2/100
20/20 - 0s - loss: 0.6876 - accuracy: 0.7150 - val_loss: 0.6855 - val_accuracy: 0.7175 - 76ms/epoch - 4ms/step
Epoch 3/100
20/20 - 0s - loss: 0.6839 - accuracy: 0.7150 - val_loss: 0.6820 - val_accuracy: 0.7175 - 79ms/epoch - 4ms/step
Epoch 4/100
20/20 - 0s - loss: 0.6804 - accuracy: 0.7150 - val_loss: 0.6785 - val_accuracy: 0.7175 - 85ms/epoch - 4ms/step
Epoch 5/100
20/20 - 0s - loss: 0.6770 - accuracy: 0.7150 - val_loss: 0.6752 - val_accuracy: 0.7175 - 65ms/epoch - 3ms/step
Epoch 6/100
20/20 - 0s - loss: 0.6738 - accuracy: 0.7150 - val_loss: 0.6718 - val_accuracy: 0.7175 - 76ms/epoch - 4ms/step
Epoch 7/100
20/20 - 0s - loss: 0.6705 - accuracy: 0.7150 - val_loss: 0.6686 - val_accuracy: 0.7175 - 70ms/epoch - 3ms/step
Epoch 8/100
20/20 - 0s - loss: 0.6675 - accuracy: 0.7150 - val_loss: 0.6655 - val_accuracy: 0.7175 - 70ms/epoch - 4ms/step
Epoch 9/100
20

Epoch 68/100
20/20 - 0s - loss: 0.6005 - accuracy: 0.7150 - val_loss: 0.5986 - val_accuracy: 0.7175 - 55ms/epoch - 3ms/step
Epoch 69/100
20/20 - 0s - loss: 0.6004 - accuracy: 0.7150 - val_loss: 0.5984 - val_accuracy: 0.7175 - 52ms/epoch - 3ms/step
Epoch 70/100
20/20 - 0s - loss: 0.6002 - accuracy: 0.7150 - val_loss: 0.5983 - val_accuracy: 0.7175 - 50ms/epoch - 2ms/step
Epoch 71/100
20/20 - 0s - loss: 0.6000 - accuracy: 0.7150 - val_loss: 0.5981 - val_accuracy: 0.7175 - 55ms/epoch - 3ms/step
Epoch 72/100
20/20 - 0s - loss: 0.5999 - accuracy: 0.7150 - val_loss: 0.5979 - val_accuracy: 0.7175 - 59ms/epoch - 3ms/step
Epoch 73/100
20/20 - 0s - loss: 0.5998 - accuracy: 0.7150 - val_loss: 0.5977 - val_accuracy: 0.7175 - 56ms/epoch - 3ms/step
Epoch 74/100
20/20 - 0s - loss: 0.5996 - accuracy: 0.7150 - val_loss: 0.5976 - val_accuracy: 0.7175 - 50ms/epoch - 3ms/step
Epoch 75/100
20/20 - 0s - loss: 0.5995 - accuracy: 0.7150 - val_loss: 0.5975 - val_accuracy: 0.7175 - 52ms/epoch - 3ms/step
Epoch 76

## Task 2. Classify all days of the week
Train a neural network against the origial categorical label. Use 5 layers of 40,15,5 and 7 (outputs, representing probabilities for a current input to correspond to each of the weekdays) neurons over the training sample, evaluating its performance over the test sample (use 'categorical_accurary'). Report the acheived accuracy (categorical) over the test sample.

First three layers use relu activation function, last one - sigmoid.
Use loss='binary_crossentropy', optimizer='adam', 200 epochs, batch_size=20

In [95]:
X_test=X[400:,:]; X_train=X[:400,:]; #split the data into training and test
y_test=yc[400:,:]; y_train=yc[:400,:]

In [96]:
dim = X_train.shape[1]
model = Sequential()
model.add(Dense(40, activation='relu', input_dim=dim))
model.add(Dense(15, activation='relu'))
model.add(Dense(5, activation='relu'))
model.add(Dense(7, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=200, batch_size=20, verbose=2)

Epoch 1/200
20/20 - 5s - loss: 0.6368 - accuracy: 0.2750 - val_loss: 0.6070 - val_accuracy: 0.2714 - 5s/epoch - 259ms/step
Epoch 2/200
20/20 - 0s - loss: 0.5837 - accuracy: 0.2800 - val_loss: 0.5581 - val_accuracy: 0.2639 - 83ms/epoch - 4ms/step
Epoch 3/200
20/20 - 0s - loss: 0.5365 - accuracy: 0.2825 - val_loss: 0.5103 - val_accuracy: 0.2751 - 60ms/epoch - 3ms/step
Epoch 4/200
20/20 - 0s - loss: 0.4875 - accuracy: 0.2950 - val_loss: 0.4637 - val_accuracy: 0.2751 - 76ms/epoch - 4ms/step
Epoch 5/200
20/20 - 0s - loss: 0.4440 - accuracy: 0.2900 - val_loss: 0.4221 - val_accuracy: 0.2788 - 67ms/epoch - 3ms/step
Epoch 6/200
20/20 - 0s - loss: 0.4059 - accuracy: 0.2775 - val_loss: 0.3913 - val_accuracy: 0.2751 - 73ms/epoch - 4ms/step
Epoch 7/200
20/20 - 0s - loss: 0.3804 - accuracy: 0.2800 - val_loss: 0.3716 - val_accuracy: 0.2825 - 65ms/epoch - 3ms/step
Epoch 8/200
20/20 - 0s - loss: 0.3643 - accuracy: 0.2775 - val_loss: 0.3567 - val_accuracy: 0.2751 - 56ms/epoch - 3ms/step
Epoch 9/200
20/2

Epoch 68/200
20/20 - 0s - loss: 0.2158 - accuracy: 0.6600 - val_loss: 0.2444 - val_accuracy: 0.6617 - 56ms/epoch - 3ms/step
Epoch 69/200
20/20 - 0s - loss: 0.2146 - accuracy: 0.6725 - val_loss: 0.2582 - val_accuracy: 0.6059 - 55ms/epoch - 3ms/step
Epoch 70/200
20/20 - 0s - loss: 0.2178 - accuracy: 0.6100 - val_loss: 0.2523 - val_accuracy: 0.6431 - 55ms/epoch - 3ms/step
Epoch 71/200
20/20 - 0s - loss: 0.2162 - accuracy: 0.6600 - val_loss: 0.2593 - val_accuracy: 0.5799 - 54ms/epoch - 3ms/step
Epoch 72/200
20/20 - 0s - loss: 0.2152 - accuracy: 0.6525 - val_loss: 0.2438 - val_accuracy: 0.6654 - 66ms/epoch - 3ms/step
Epoch 73/200
20/20 - 0s - loss: 0.2113 - accuracy: 0.6725 - val_loss: 0.2448 - val_accuracy: 0.6803 - 53ms/epoch - 3ms/step
Epoch 74/200
20/20 - 0s - loss: 0.2096 - accuracy: 0.6600 - val_loss: 0.2475 - val_accuracy: 0.6840 - 56ms/epoch - 3ms/step
Epoch 75/200
20/20 - 0s - loss: 0.2057 - accuracy: 0.6950 - val_loss: 0.2422 - val_accuracy: 0.6543 - 56ms/epoch - 3ms/step
Epoch 76

Epoch 134/200
20/20 - 0s - loss: 0.1673 - accuracy: 0.7650 - val_loss: 0.2361 - val_accuracy: 0.6840 - 54ms/epoch - 3ms/step
Epoch 135/200
20/20 - 0s - loss: 0.1679 - accuracy: 0.7675 - val_loss: 0.2413 - val_accuracy: 0.6394 - 70ms/epoch - 4ms/step
Epoch 136/200
20/20 - 0s - loss: 0.1688 - accuracy: 0.7450 - val_loss: 0.2639 - val_accuracy: 0.5874 - 49ms/epoch - 2ms/step
Epoch 137/200
20/20 - 0s - loss: 0.1667 - accuracy: 0.7800 - val_loss: 0.2528 - val_accuracy: 0.6134 - 49ms/epoch - 2ms/step
Epoch 138/200
20/20 - 0s - loss: 0.1664 - accuracy: 0.7650 - val_loss: 0.2683 - val_accuracy: 0.6134 - 51ms/epoch - 3ms/step
Epoch 139/200
20/20 - 0s - loss: 0.1696 - accuracy: 0.7625 - val_loss: 0.2469 - val_accuracy: 0.6468 - 49ms/epoch - 2ms/step
Epoch 140/200
20/20 - 0s - loss: 0.1697 - accuracy: 0.7550 - val_loss: 0.2585 - val_accuracy: 0.6357 - 48ms/epoch - 2ms/step
Epoch 141/200
20/20 - 0s - loss: 0.1649 - accuracy: 0.7650 - val_loss: 0.2553 - val_accuracy: 0.6208 - 48ms/epoch - 2ms/step


Epoch 200/200
20/20 - 0s - loss: 0.1425 - accuracy: 0.8075 - val_loss: 0.2532 - val_accuracy: 0.6394 - 55ms/epoch - 3ms/step


## part 2

Use the same datasets as in the Advanced NN lab and further train NN with different architectures

In [191]:

X_train_pp = (X_train.reshape(X_train.shape[0], 10,10, 1).astype('float32')) / 255 
X_test_pp = (X_test.reshape(X_test.shape[0], 10,10, 1).astype('float32')) / 255 

# Keras needs the labels as categorical metrics.
y_train_pp = y_train
y_test_pp = y_test
num_classes = y_test_pp.shape[1]

In [192]:
## model

np.random.seed(1066)
model = Sequential()
model.add(Conv2D(32, (5, 5), input_shape=(10,10, 1), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

# Compile, fit, and generate scores and predicted probabilities.
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train_pp, y_train_pp, validation_data=(X_test_pp, y_test_pp), epochs=100, batch_size=200, verbose=2)
scores = model.evaluate(X_test_pp, y_test_pp, verbose=0)
prob = model.predict(X_test_pp)

Epoch 1/100
2/2 - 1s - loss: 1.9461 - accuracy: 0.1400 - val_loss: 1.9457 - val_accuracy: 0.1450 - 687ms/epoch - 344ms/step
Epoch 2/100
2/2 - 0s - loss: 1.9457 - accuracy: 0.1825 - val_loss: 1.9457 - val_accuracy: 0.1747 - 45ms/epoch - 23ms/step
Epoch 3/100
2/2 - 0s - loss: 1.9457 - accuracy: 0.1900 - val_loss: 1.9456 - val_accuracy: 0.1413 - 39ms/epoch - 20ms/step
Epoch 4/100
2/2 - 0s - loss: 1.9456 - accuracy: 0.2075 - val_loss: 1.9455 - val_accuracy: 0.1413 - 35ms/epoch - 17ms/step
Epoch 5/100
2/2 - 0s - loss: 1.9455 - accuracy: 0.1650 - val_loss: 1.9455 - val_accuracy: 0.2230 - 35ms/epoch - 18ms/step
Epoch 6/100
2/2 - 0s - loss: 1.9455 - accuracy: 0.2000 - val_loss: 1.9454 - val_accuracy: 0.2788 - 36ms/epoch - 18ms/step
Epoch 7/100
2/2 - 0s - loss: 1.9454 - accuracy: 0.1850 - val_loss: 1.9453 - val_accuracy: 0.1784 - 38ms/epoch - 19ms/step
Epoch 8/100
2/2 - 0s - loss: 1.9455 - accuracy: 0.1650 - val_loss: 1.9453 - val_accuracy: 0.1487 - 35ms/epoch - 17ms/step
Epoch 9/100
2/2 - 0s -

Epoch 68/100
2/2 - 0s - loss: 1.8457 - accuracy: 0.2825 - val_loss: 1.8433 - val_accuracy: 0.2714 - 33ms/epoch - 17ms/step
Epoch 69/100
2/2 - 0s - loss: 1.8403 - accuracy: 0.2825 - val_loss: 1.8376 - val_accuracy: 0.2714 - 33ms/epoch - 16ms/step
Epoch 70/100
2/2 - 0s - loss: 1.8351 - accuracy: 0.2825 - val_loss: 1.8316 - val_accuracy: 0.2751 - 31ms/epoch - 16ms/step
Epoch 71/100
2/2 - 0s - loss: 1.8255 - accuracy: 0.2825 - val_loss: 1.8252 - val_accuracy: 0.2714 - 32ms/epoch - 16ms/step
Epoch 72/100
2/2 - 0s - loss: 1.8202 - accuracy: 0.2825 - val_loss: 1.8183 - val_accuracy: 0.2714 - 31ms/epoch - 15ms/step
Epoch 73/100
2/2 - 0s - loss: 1.8119 - accuracy: 0.2775 - val_loss: 1.8115 - val_accuracy: 0.2714 - 31ms/epoch - 16ms/step
Epoch 74/100
2/2 - 0s - loss: 1.8078 - accuracy: 0.2775 - val_loss: 1.8045 - val_accuracy: 0.2714 - 32ms/epoch - 16ms/step
Epoch 75/100
2/2 - 0s - loss: 1.7995 - accuracy: 0.2800 - val_loss: 1.7979 - val_accuracy: 0.2714 - 29ms/epoch - 15ms/step
Epoch 76/100
2/2

In [187]:

# X_train_pp = (X_train.reshape(X_train.shape[0], 4,25, 1).astype('float32')) / 255 
# X_test_pp = (X_test.reshape(X_test.shape[0], 4,25,  1).astype('float32')) / 255 

# # Keras needs the labels as categorical metrics.
# y_train_pp = y_train
# y_test_pp = y_test
# num_classes = y_test_pp.shape[1]

In [196]:
np.random.seed(2002)
model = Sequential()
model.add(Conv2D(32, (5, 5), input_shape=(10, 10, 1), activation='relu'))
model.add(Conv2D(32, (5, 5), input_shape=(10, 1), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Conv2D(32, (5, 5), input_shape=(10,10, 1), activation='relu'))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

# Compile, fit, and generate scores and predicted probabilities.
# should take under 2-3 minutes to train

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train_pp, y_train_pp, validation_data=(X_test_pp, y_test_pp), epochs=50, batch_size=200, verbose=2)
scores = model.evaluate(X_test_pp, y_test_pp, verbose=0)
prob = model.predict(X_test_pp)

Epoch 1/50
2/2 - 1s - loss: 1.9460 - accuracy: 0.1525 - val_loss: 1.9458 - val_accuracy: 0.1450 - 601ms/epoch - 300ms/step
Epoch 2/50
2/2 - 0s - loss: 1.9460 - accuracy: 0.1425 - val_loss: 1.9458 - val_accuracy: 0.1450 - 38ms/epoch - 19ms/step
Epoch 3/50
2/2 - 0s - loss: 1.9459 - accuracy: 0.1425 - val_loss: 1.9458 - val_accuracy: 0.1450 - 39ms/epoch - 20ms/step
Epoch 4/50
2/2 - 0s - loss: 1.9458 - accuracy: 0.1450 - val_loss: 1.9457 - val_accuracy: 0.1450 - 41ms/epoch - 20ms/step
Epoch 5/50
2/2 - 0s - loss: 1.9458 - accuracy: 0.1450 - val_loss: 1.9457 - val_accuracy: 0.1450 - 40ms/epoch - 20ms/step
Epoch 6/50
2/2 - 0s - loss: 1.9458 - accuracy: 0.1425 - val_loss: 1.9456 - val_accuracy: 0.1450 - 35ms/epoch - 17ms/step
Epoch 7/50
2/2 - 0s - loss: 1.9456 - accuracy: 0.1500 - val_loss: 1.9455 - val_accuracy: 0.2788 - 40ms/epoch - 20ms/step
Epoch 8/50
2/2 - 0s - loss: 1.9455 - accuracy: 0.2000 - val_loss: 1.9455 - val_accuracy: 0.1413 - 45ms/epoch - 23ms/step
Epoch 9/50
2/2 - 0s - loss: 1.

In [205]:
scores[1]

0.27881041169166565

## Task 1

Try the facial recognition task from NN lab 2 with different model specifications and report model accuracy:
1. Try different dropouts - 0.2, 0.3, 0.4
2. Try different convolution windows: (3$\times$3, 10$\times$10)

In [61]:
# load the data

from sklearn.datasets import fetch_lfw_people 
from keras.datasets import mnist

lfw_people = fetch_lfw_people(min_faces_per_person = 70, resize = 0.4) 
  
# the images arrays to find the shapes (for plotting) 
n_samples, h, w = lfw_people.images.shape 
  
# Instead of providing 2D data, X has data already in the form  of a vector that 
# is required in this approach. 
X = lfw_people.data 
n_features = X.shape[1] 

y = lfw_people.target 
target_names = lfw_people.target_names 
n_classes = target_names.shape[0] 

In [62]:
X.shape

(1288, 1850)

In [63]:
# lets split and reform the dataset in the format compatible for CNN
# basically we are restructuring the data into grid shapes which are apt for convolutions and further steps

X_train, X_test, y_train, y_test = train_test_split( 
    X, y, test_size = 0.2)

X_train_pp = (X_train.reshape(X_train.shape[0], 50, 37, 1).astype('float32')) / 255 # normalize
X_test_pp = (X_test.reshape(X_test.shape[0], 50, 37, 1).astype('float32')) / 255

y_train_pp = np_utils.to_categorical(y_train)
y_test_pp = np_utils.to_categorical(y_test)

num_classes = y_train_pp.shape[1]

In [64]:
print(X_train_pp.shape, X_test_pp.shape)

(1030, 50, 37, 1) (258, 50, 37, 1)


In [65]:
# let's change the dropout % to 0.4 and check the out of sample accuracy

np.random.seed(2002)
model = Sequential()
model.add(Conv2D(32, (5, 5), input_shape=(50, 37, 1), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (5, 5), input_shape=(50, 37, 1), activation='relu'))
model.add(Dropout(0.4))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

# Compile, fit, and generate scores and predicted probabilities.
# should take under 2-3 minutes to train

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train_pp, y_train_pp, validation_data=(X_test_pp, y_test_pp), epochs=50, batch_size=200, verbose=2)
scores = model.evaluate(X_test_pp, y_test_pp, verbose=0)
prob = model.predict(X_test_pp)

Epoch 1/50
6/6 - 2s - loss: 1.7705 - accuracy: 0.3583 - val_loss: 1.6371 - val_accuracy: 0.4496 - 2s/epoch - 294ms/step
Epoch 2/50
6/6 - 1s - loss: 1.6736 - accuracy: 0.4097 - val_loss: 1.6101 - val_accuracy: 0.4496 - 882ms/epoch - 147ms/step
Epoch 3/50
6/6 - 1s - loss: 1.6401 - accuracy: 0.4117 - val_loss: 1.6271 - val_accuracy: 0.4922 - 991ms/epoch - 165ms/step
Epoch 4/50
6/6 - 1s - loss: 1.5961 - accuracy: 0.4252 - val_loss: 1.5118 - val_accuracy: 0.4884 - 964ms/epoch - 161ms/step
Epoch 5/50
6/6 - 1s - loss: 1.5122 - accuracy: 0.4825 - val_loss: 1.4754 - val_accuracy: 0.4961 - 905ms/epoch - 151ms/step
Epoch 6/50
6/6 - 1s - loss: 1.4094 - accuracy: 0.5117 - val_loss: 1.3762 - val_accuracy: 0.5271 - 1s/epoch - 173ms/step
Epoch 7/50
6/6 - 1s - loss: 1.3074 - accuracy: 0.5437 - val_loss: 1.3025 - val_accuracy: 0.5814 - 1s/epoch - 178ms/step
Epoch 8/50
6/6 - 1s - loss: 1.2303 - accuracy: 0.5806 - val_loss: 1.2606 - val_accuracy: 0.5620 - 1s/epoch - 191ms/step
Epoch 9/50
6/6 - 1s - loss: 

In [72]:
from sklearn.metrics import accuracy_score
predict_x=model.predict(X_test_pp) 
classes_x=np.argmax(predict_x,axis=1)

In [74]:
#0.4
accuracy_score(y_test, classes_x)

0.8837209302325582

In [75]:
# let's change the dropout % to 0.3 and check the out of sample accuracy

np.random.seed(2002)
model = Sequential()
model.add(Conv2D(32, (5, 5), input_shape=(50, 37, 1), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (5, 5), input_shape=(50, 37, 1), activation='relu'))
model.add(Dropout(0.3))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

# Compile, fit, and generate scores and predicted probabilities.
# should take under 2-3 minutes to train

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train_pp, y_train_pp, validation_data=(X_test_pp, y_test_pp), epochs=50, batch_size=200, verbose=2)
scores = model.evaluate(X_test_pp, y_test_pp, verbose=0)
prob = model.predict(X_test_pp)

Epoch 1/50
6/6 - 2s - loss: 1.8667 - accuracy: 0.3039 - val_loss: 1.7640 - val_accuracy: 0.4496 - 2s/epoch - 258ms/step
Epoch 2/50
6/6 - 1s - loss: 1.7327 - accuracy: 0.4019 - val_loss: 1.6401 - val_accuracy: 0.4496 - 937ms/epoch - 156ms/step
Epoch 3/50
6/6 - 1s - loss: 1.6934 - accuracy: 0.4019 - val_loss: 1.6568 - val_accuracy: 0.4496 - 931ms/epoch - 155ms/step
Epoch 4/50
6/6 - 1s - loss: 1.6733 - accuracy: 0.4019 - val_loss: 1.6353 - val_accuracy: 0.4496 - 957ms/epoch - 160ms/step
Epoch 5/50
6/6 - 1s - loss: 1.6619 - accuracy: 0.4019 - val_loss: 1.6252 - val_accuracy: 0.4496 - 946ms/epoch - 158ms/step
Epoch 6/50
6/6 - 1s - loss: 1.6530 - accuracy: 0.4019 - val_loss: 1.6192 - val_accuracy: 0.4496 - 913ms/epoch - 152ms/step
Epoch 7/50
6/6 - 1s - loss: 1.6271 - accuracy: 0.4019 - val_loss: 1.6121 - val_accuracy: 0.4496 - 1000ms/epoch - 167ms/step
Epoch 8/50
6/6 - 1s - loss: 1.6105 - accuracy: 0.4019 - val_loss: 1.5635 - val_accuracy: 0.4496 - 1s/epoch - 176ms/step
Epoch 9/50
6/6 - 1s -

In [76]:
#0.3
predict_x=model.predict(X_test_pp) 
classes_x=np.argmax(predict_x,axis=1)
accuracy_score(y_test, classes_x)

0.8604651162790697

In [78]:
# let's change the dropout % to 0.2 and check the out of sample accuracy

np.random.seed(2002)
model = Sequential()
model.add(Conv2D(32, (5, 5), input_shape=(50, 37, 1), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (5, 5), input_shape=(50, 37, 1), activation='relu'))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

# Compile, fit, and generate scores and predicted probabilities.
# should take under 2-3 minutes to train

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train_pp, y_train_pp, validation_data=(X_test_pp, y_test_pp), epochs=50, batch_size=200, verbose=2)
scores = model.evaluate(X_test_pp, y_test_pp, verbose=0)
prob = model.predict(X_test_pp)

Epoch 1/50
6/6 - 1s - loss: 1.7991 - accuracy: 0.3447 - val_loss: 1.6491 - val_accuracy: 0.4496 - 1s/epoch - 245ms/step
Epoch 2/50
6/6 - 1s - loss: 1.6851 - accuracy: 0.4019 - val_loss: 1.6391 - val_accuracy: 0.4496 - 909ms/epoch - 151ms/step
Epoch 3/50
6/6 - 1s - loss: 1.6727 - accuracy: 0.4019 - val_loss: 1.6302 - val_accuracy: 0.4496 - 977ms/epoch - 163ms/step
Epoch 4/50
6/6 - 1s - loss: 1.6440 - accuracy: 0.4019 - val_loss: 1.5892 - val_accuracy: 0.4496 - 1s/epoch - 174ms/step
Epoch 5/50
6/6 - 1s - loss: 1.6018 - accuracy: 0.4029 - val_loss: 1.6266 - val_accuracy: 0.4690 - 935ms/epoch - 156ms/step
Epoch 6/50
6/6 - 1s - loss: 1.6152 - accuracy: 0.4029 - val_loss: 1.5299 - val_accuracy: 0.4806 - 912ms/epoch - 152ms/step
Epoch 7/50
6/6 - 1s - loss: 1.5286 - accuracy: 0.4534 - val_loss: 1.4626 - val_accuracy: 0.4845 - 928ms/epoch - 155ms/step
Epoch 8/50
6/6 - 1s - loss: 1.4460 - accuracy: 0.5175 - val_loss: 1.3766 - val_accuracy: 0.5078 - 941ms/epoch - 157ms/step
Epoch 9/50
6/6 - 1s - 

In [79]:
#0.2
predict_x=model.predict(X_test_pp) 
classes_x=np.argmax(predict_x,axis=1)
accuracy_score(y_test, classes_x)

0.8953488372093024

In [80]:
#Try different convolution windows:3 × 3

np.random.seed(2002)
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(50, 37, 1), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), input_shape=(50, 37, 1), activation='relu'))
model.add(Dropout(0.4))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

# Compile, fit, and generate scores and predicted probabilities.
# should take under 2-3 minutes to train

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train_pp, y_train_pp, validation_data=(X_test_pp, y_test_pp), epochs=50, batch_size=200, verbose=2)
scores = model.evaluate(X_test_pp, y_test_pp, verbose=0)
prob = model.predict(X_test_pp)

Epoch 1/50
6/6 - 1s - loss: 1.9069 - accuracy: 0.2835 - val_loss: 1.7201 - val_accuracy: 0.4496 - 1s/epoch - 219ms/step
Epoch 2/50
6/6 - 1s - loss: 1.7160 - accuracy: 0.4019 - val_loss: 1.6263 - val_accuracy: 0.4496 - 805ms/epoch - 134ms/step
Epoch 3/50
6/6 - 1s - loss: 1.6503 - accuracy: 0.4019 - val_loss: 1.5966 - val_accuracy: 0.4496 - 787ms/epoch - 131ms/step
Epoch 4/50
6/6 - 1s - loss: 1.6003 - accuracy: 0.4194 - val_loss: 1.5276 - val_accuracy: 0.4651 - 795ms/epoch - 132ms/step
Epoch 5/50
6/6 - 1s - loss: 1.5147 - accuracy: 0.4621 - val_loss: 1.4515 - val_accuracy: 0.5194 - 831ms/epoch - 139ms/step
Epoch 6/50
6/6 - 1s - loss: 1.4275 - accuracy: 0.5000 - val_loss: 1.3760 - val_accuracy: 0.5310 - 796ms/epoch - 133ms/step
Epoch 7/50
6/6 - 1s - loss: 1.2981 - accuracy: 0.5330 - val_loss: 1.2743 - val_accuracy: 0.5388 - 788ms/epoch - 131ms/step
Epoch 8/50
6/6 - 1s - loss: 1.2245 - accuracy: 0.5728 - val_loss: 1.2019 - val_accuracy: 0.5620 - 808ms/epoch - 135ms/step
Epoch 9/50
6/6 - 1s

In [81]:
#3x3
predict_x=model.predict(X_test_pp) 
classes_x=np.argmax(predict_x,axis=1)
accuracy_score(y_test, classes_x)

0.9108527131782945

In [82]:
#Try different convolution windows:10 × 10

np.random.seed(2002)
model = Sequential()
model.add(Conv2D(32, (10, 10), input_shape=(50, 37, 1), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (10, 10), input_shape=(50, 37, 1), activation='relu'))
model.add(Dropout(0.4))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

# Compile, fit, and generate scores and predicted probabilities.
# should take under 2-3 minutes to train

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train_pp, y_train_pp, validation_data=(X_test_pp, y_test_pp), epochs=50, batch_size=200, verbose=2)
scores = model.evaluate(X_test_pp, y_test_pp, verbose=0)
prob = model.predict(X_test_pp)

Epoch 1/50
6/6 - 2s - loss: 1.8074 - accuracy: 0.2592 - val_loss: 1.7207 - val_accuracy: 0.4496 - 2s/epoch - 251ms/step
Epoch 2/50
6/6 - 1s - loss: 1.7144 - accuracy: 0.4010 - val_loss: 1.7222 - val_accuracy: 0.4496 - 934ms/epoch - 156ms/step
Epoch 3/50
6/6 - 1s - loss: 1.7546 - accuracy: 0.4019 - val_loss: 1.6462 - val_accuracy: 0.4496 - 921ms/epoch - 154ms/step
Epoch 4/50
6/6 - 1s - loss: 1.6932 - accuracy: 0.4019 - val_loss: 1.6700 - val_accuracy: 0.4496 - 941ms/epoch - 157ms/step
Epoch 5/50
6/6 - 1s - loss: 1.7043 - accuracy: 0.4019 - val_loss: 1.6582 - val_accuracy: 0.4496 - 928ms/epoch - 155ms/step
Epoch 6/50
6/6 - 1s - loss: 1.6820 - accuracy: 0.4019 - val_loss: 1.6516 - val_accuracy: 0.4496 - 875ms/epoch - 146ms/step
Epoch 7/50
6/6 - 1s - loss: 1.6794 - accuracy: 0.4019 - val_loss: 1.6458 - val_accuracy: 0.4496 - 969ms/epoch - 162ms/step
Epoch 8/50
6/6 - 1s - loss: 1.6850 - accuracy: 0.4019 - val_loss: 1.6634 - val_accuracy: 0.4496 - 957ms/epoch - 160ms/step
Epoch 9/50
6/6 - 1s

In [83]:
#10x10
predict_x=model.predict(X_test_pp) 
classes_x=np.argmax(predict_x,axis=1)
accuracy_score(y_test, classes_x)

0.7945736434108527